# LTSM SIMPLE



## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.



In [0]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data('non-secuencial')

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mostrando los datos de data_total_for_non_serial.csv
(30555, 33)
________________________________________________________________________________


In [0]:
data_consumo.columns

Index(['fecha', 'PVPC_DEF', 'PVPC_2_PED_NOC', 'PVPC_ELEC_NOC',
       'date_timestamp', 'Demanda', 'Eolica', 'Nuclear', 'Solar',
       'Solar_Fotovoltaica', 'Solar_Termica', 'Termica_Renovable', 'Prevista',
       'Programada', 'date_day', 'Brent_price', 'Holiday',
       'Precio de Regulación Secundaria subir',
       'Precio de Regulación Secundaria bajar', 'Precio mercado SPOT Diario_x',
       'Demanda real', 'Generación prevista Solar',
       'Saldo total interconexiones programa p48',
       'Generación programada P48 Exportación Portugal',
       'Generación programada P48 Exportación Francia',
       'Generación programada P48 Importación Portugal',
       'Generación programada P48 Importación Francia', 'Precio SPOT PT',
       'Precio SPOT FR', 'PVPC-1h', 'PVPC-2h', 'PVPC-1d', 'PVPC-1w'],
      dtype='object')

## Preparación de los datos

Sobre el tema de los datos, nosotros utilizamos los datos a tiempo real sin los datos de precio que no son disponibles. El objetivo es predecir el precio a d+1.

In [0]:
x_data = esios_controller.get_data_real_time()
y_data = esios_controller.get_target_data()


x_data = x_data.drop(['PVPC-1h', 'PVPC-2h'], axis=1)

x_data.rename(columns={'date_timestamp':'timestamp'},  inplace=True)
y_data.rename(columns={'date_timestamp':'timestamp'},  inplace=True)
x_data.head()

,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,PVPC-1d,PVPC-1w
0,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,47.48,0.0,61.18,49.13,267.6,182.4,-606.6,-351.4,1232.1,1200.0,0.0,0.0
1,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,47.48,0.0,59.59,49.13,195.7,-269.5,-1112.5,-351.4,1232.1,1200.0,0.0,0.0
2,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,47.48,0.0,56.27,49.13,202.1,94.0,-951.0,-351.4,1232.1,1200.0,0.0,0.0
3,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,47.48,0.0,45.99,45.50,61.3,319.7,-846.3,-351.4,1232.1,1200.0,0.0,0.0
4,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,47.48,0.0,52.28,49.13,45.1,405.5,-760.5,-351.4,1232.1,1200.0,0.0,0.0


In [0]:
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

#Convert NaN values to 0:
x_train = np.nan_to_num(x_train)
x_valid = np.nan_to_num(x_valid)
y_train = np.nan_to_num(y_train)
y_valid = np.nan_to_num(y_valid)

print('Xtrain_dim:', x_train.shape)
print('Xvalid_dim:', x_valid.shape)
print('Ytrain_dim:', y_train.shape)
print('Yvalid_dim:', y_valid.shape)

Xtrain_dim: (20471, 21)
Xvalid_dim: (10084, 21)
Ytrain_dim: (20471, 1)
Yvalid_dim: (10084, 1)


In [0]:
print(y_train)

[[117.77]
 [117.09]
 [114.59]
 ...
 [121.99]
 [119.9 ]
 [119.36]]


In [0]:
def rmse (y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred -y_true), axis=-1))


def r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

## Modelo

In [0]:
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import SGD
from keras.regularizers import l2

batch_size = 16
num_steps = 20
num_var = x_train.shape[1]
hidden_size = 10
output_size = 1

In [0]:
# split a sequence into samples
def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
        break
      # gather input and output parts of the pattern
      seq_x = sequence[i:end_ix,:]
      X.append(seq_x)

    return np.array(X)

In [0]:
scaler = MinMaxScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)

x_train_data = split_sequence(x_train_est, num_steps)
y_train_data = y_train_est[num_steps:]
x_valid_data = split_sequence(x_valid, num_steps)
y_valid_data = y_valid[num_steps:]

print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
print('Test:',x_valid_data.shape, y_valid_data.shape)

Entrenamiento: (20451, 20, 21) (20451, 1)
Test: (10064, 20, 21) (10064, 1)


In [0]:
# define model
model = Sequential()
model.add(LSTM(hidden_size, 
             activation='relu', 
             return_sequences=True, 
             kernel_initializer='zeros',
             input_shape=(num_steps,num_var),
             kernel_regularizer=l2(1e-2)))
model.add(Dropout(0.4))
model.add(LSTM(hidden_size, 
             activation='relu',
             kernel_initializer='zeros',
             kernel_regularizer=l2(1e-2)))
model.add(Dropout(0.5))
model.add(Dense(1,
             activation='linear',
             kernel_regularizer=l2(1e-2)))

# compile mode
opt = SGD(lr=0.1, momentum=0.9, decay= 1e-2, nesterov=True)
model.compile(optimizer=opt, loss='mse', metrics=['mse',rmse,r2,'mae','mape'])
print(model.summary())

W0821 17:48:18.136612 140038669047680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 17:48:18.158796 140038669047680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 17:48:18.379911 140038669047680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0821 17:48:18.391745 140038669047680 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a futur

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 10)            1280      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 10)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                840       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 2,131
Trainable params: 2,131
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
history_train = model.fit(x_train_data, y_train_data, 
                        epochs=100, 
                        batch_size=batch_size, 
                        shuffle=False,
                        verbose=1)

# summarize history for loss, MAPE in a different scale than the others
plt.plot(history_train.history['mean_squared_error'])
plt.plot(history_train.history['mean_absolute_error'])
plt.plot(history_train.history['rmse'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['MSE','MAE','RMSE'], loc='upper left')
plt.show()

plt.plot(history_train.history['mean_absolute_percentage_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['MAPE'], loc='upper left')
plt.show()

Epoch 1/100
20451/20451 [==============================] - 108s 5ms/step - loss: 0.0107 - mean_squared_error: 0.0107 - rmse: 0.0761 - r2: -5.6727 - mean_absolute_error: 0.0761 - mean_absolute_percentage_error: 14820.6798
Epoch 2/100
20451/20451 [==============================] - 107s 5ms/step - loss: 0.0111 - mean_squared_error: 0.0111 - rmse: 0.0778 - r2: -5.9566 - mean_absolute_error: 0.0778 - mean_absolute_percentage_error: 15385.2729
Epoch 3/100
20451/20451 [==============================] - 108s 5ms/step - loss: 0.0115 - mean_squared_error: 0.0115 - rmse: 0.0792 - r2: -6.1988 - mean_absolute_error: 0.0792 - mean_absolute_percentage_error: 15889.9086
Epoch 4/100
 4976/20451 [======>.......................] - ETA: 1:22 - loss: 0.0152 - mean_squared_error: 0.0152 - rmse: 0.0964 - r2: -8.0881 - mean_absolute_error: 0.0964 - mean_absolute_percentage_error: 67049.3460

KeyboardInterrupt: ignored

## Normalizados

In [0]:
scaler = StandardScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)
x_valid_est = scaler.fit_transform(x_valid)
y_valid_est = scaler.fit_transform(y_valid)

In [0]:
regressor_standarized = DecisionTreeRegressor(random_state=0)
regressor_standarized.fit(x_train_est, y_train_est)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

In [0]:
print('R2 en entrenamiento es: ', regressor_standarized.score(x_train_est, y_train_est))
print('R2 en validación es: ', regressor_standarized.score(x_valid_est, y_valid_est))

print('MAE: ', mean_absolute_error(regressor_standarized.predict(x_valid_est), y_valid_est))
print('MSE: ', mean_squared_error(regressor_standarized.predict(x_valid_est), y_valid_est))
#print('RMSE: ', mean_squared_log_error(regressor_standarized.predict(x_valid_est), y_valid_est))
print('Variance: ', explained_variance_score(regressor_standarized.predict(x_valid_est), y_valid_est))
print('R2: ', r2_score(regressor_standarized.predict(x_valid_est), y_valid_est))

R2 en entrenamiento es:  0.9999999971664847
R2 en validación es:  0.8564415424845535
MAE:  0.2715218174119526
MSE:  0.14355845751544644
Variance:  0.8473509162286471
R2:  0.8472343601302845


## Optimización de modelos

In [0]:
param_grid = {"criterion": ["mse", "mae"],
              "min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }

clf = GridSearchCV(regressor, param_grid, cv=5)
clf.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=0,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['mse', 'mae'], 'max_depth': [2, 6, 8],
                         'max_leaf_nodes': [5, 20, 100],
                         'min_samples_leaf': [20, 40, 100],
                    

In [0]:
print(clf.best_params_)


{'criterion': 'mse', 'max_depth': 8, 'max_leaf_nodes': 100, 'min_samples_leaf': 40, 'min_samples_split': 10}


## Try best model

In [0]:
regressor = DecisionTreeRegressor(criterion='mse', max_depth= 8, max_leaf_nodes= 200, min_samples_leaf= 40, min_samples_split= 2)
model_fit = regressor.fit(x_train, y_train)

In [0]:
from sklearn.model_selection import cross_val_score

cross_val_score(regressor, x_valid, y_valid, cv=10)

array([0.89486021, 0.72979586, 0.80117989, 0.81741218, 0.76559692,
       0.88774078, 0.83484429, 0.86445979, 0.83767378, 0.71632333])

In [0]:
print('R2 en entrenamiento es: ', regressor.score(x_train, y_train))
print('R2 en validación es: ', regressor.score(x_valid, y_valid))

print('MAE: ', mean_absolute_error(regressor.predict(x_valid), y_valid))
print('MSE: ', mean_squared_error(regressor.predict(x_valid), y_valid))
#print('RMSE: ', mean_squared_log_error(regressor.predict(x_valid), y_valid))
print('Variance: ', explained_variance_score(regressor.predict(x_valid), y_valid))
print('R2: ', r2_score(regressor.predict(x_valid), y_valid))
#print(regressor.coef_)

R2 en entrenamiento es:  0.949767732603002
R2 en validación es:  0.9163412188583834
MAE:  2.3705676905981328
MSE:  10.592243073911355
Variance:  0.9072843005214764
R2:  0.905683241767359
